In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#from prob2utils_skeleton import train_model, get_err
from surprise import AlgoBase
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import matrix_factorization
from surprise import accuracy
from surprise.model_selection import train_test_split

In [4]:
movies = pd.read_table('data/movies.txt',names=["Movie Id", "Movie Title", "Unknown", "Action", "Adventure", "Animation", "Childrens", "Comedy", "Crime", "Documentary","Drama", "Fantasy", "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"])
np.save("movie", movies)

data = pd.read_table('data/data.txt', names=["user", "movie", "rating"])
np.save("data", data)

ytrain = pd.read_table('data/train.txt',  names=["user", "movie", "rating"])
np.save("train", ytrain)

test = pd.read_table('data/test.txt', names=["user", "movie", "rating"])
np.save("test", test)

In [7]:
reader = Reader(rating_scale=(1, 5))
ydata = Dataset.load_from_df(data, reader=reader)
trainset, testset = train_test_split(ydata, test_size=.25)

In [8]:
filterer = matrix_factorization.SVD()
filterer.fit(trainset)

In [9]:
np.savetxt('u_shelf.txt', filterer.pu)
np.savetxt('v_shelf.txt', filterer.qi)
np.savetxt('ubias_shelf.txt', filterer.bu)
np.savetxt('vbias_shelf.txt', filterer.bi)